TO DO
- Develop code to handle no game / double game gameweeks (see, e.g. funtion get_was_home)

In [ ]:
# number of latest gameweek
gameweek = 38
# check number of total players from fpl's site
total_players = 11447257

In [ ]:
import pandas as pd
import numpy as np

import json
import requests
import sys

# allow more data columns to be shown than by default
pd.set_option('display.max_columns', 500)

In [ ]:
sys.path.append('..')

In [ ]:
import py_scripts.download_data as download_data

# Fetch FPL data

In [ ]:
# for some reason, fpl team numbers are otherwise alphabetic, except Leeds comes after Leicester...
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
        'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leicester City', 'Leeds United',
         'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham','Wolves']

In [ ]:
# fetch FPL data online
fpl_orig_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_data = pd.DataFrame(fpl_orig_data['elements'])
#fpl_data.set_index('id',inplace=True)
fpl_data['season'] = '2022-23'
fpl_data['name'] = fpl_data['first_name'] + ' ' + fpl_data['second_name']
# normalize spelling of names
fpl_data['name'] = fpl_data['name'].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
#fpl_data['name'] = fpl_data['name'].map(player_name_dict).fillna(fpl_data['name'])
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
fpl_data['position'] = fpl_data['element_type'].map(position_dict)
# change team number to team name
fpl_data['team'] = [teams[i-1] for i in fpl_data['team']]
fpl_data['GW'] = gameweek
fpl_data['value'] = fpl_data['now_cost']/10.0
fpl_data['selected'] = (fpl_data['selected_by_percent'].astype(float) / 100) * total_players
fpl_data['transfers_balance'] = fpl_data['transfers_in_event'] - fpl_data['transfers_out_event']
fpl_data

In [ ]:
# get FPL fixtures data
fpl_fixtures_data = json.loads(requests.get('https://fantasy.premierleague.com/api/fixtures/').text)
fpl_fixtures = pd.DataFrame(fpl_fixtures_data)
fpl_fixtures['home_team'] = [teams[i] for i in fpl_fixtures['team_h']-1]
fpl_fixtures['away_team'] = [teams[i] for i in fpl_fixtures['team_a']-1]
fpl_fixtures

In [ ]:
def get_opponent(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    # my_team is the home team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team), \
                                         'away_team']
    # my_team is the away team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team), \
                                         'home_team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) )].shape[0]!=1:
        player = my_row['name']
        print(f'Number of games not equal to one for {player} ({my_team})')
        return np.nan

    return opponent_team.values[0]

In [ ]:
def get_kickoff_time(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    kickoff_time = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) ), 'kickoff_time']
    if kickoff_time.shape[0]==1:
        return kickoff_time.values[0]
    else:
        player = my_row['name']
        print(f'Kickoff_time shape not one for {player} ({my_team})')
        return np.nan

In [ ]:
def get_was_home(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        return 1
    elif fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        return 0
    else:
        player = my_row['name']
        print(f'Error in get_was_home for {player} ({my_team})')
        return np.nan

In [ ]:
fpl_data['opp_team_name'] = fpl_data.apply(lambda x: get_opponent(x, fpl_fixtures, gameweek), axis=1)
fpl_data['kickoff_time'] = fpl_data.apply(lambda x: get_kickoff_time(x, fpl_fixtures, gameweek), axis=1)
fpl_data['was_home'] = fpl_data.apply(lambda x: get_was_home(x, fpl_fixtures, gameweek), axis=1)
fpl_data

In [ ]:
fpl_data.to_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv')
fpl_fixtures.to_csv('../data/fpl_fixtures.csv')

In [ ]:
difference_columns = [
    'total_points', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 
    'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals', 'penalties_missed',
    'penalties_saved', 'saves', 'threat', 'yellow_cards', 'red_cards'
]

In [ ]:
def calculate_difference_to_previous_gameweek(my_row):
    my_id = my_row['id']
    if my_id in old_ids:
        for my_column in difference_columns:
            my_row[my_column] = (my_row[my_column] -
                                     fpl_previous.loc[fpl_previous['id']==my_id, my_column]).values[0]
        return my_row
    elif my_id in new_ids:
        # for new players, these should be the same already, but just in case
        my_row['total_points'] = my_row['event_points']
        return my_row
    else:
        print(f'Player id neither old or new: {my_id}')
        return np.nan

In [ ]:
# create a csv that contains each week's fpl data in one table
if gameweek==1:
    fpl_data['total_points'] = fpl_data['event_points']
    fpl_data[fpl_data['minutes']>0].to_csv('../data/fpl_data.csv')
    print(f'Gameweek {gameweek} data saved.')
elif gameweek>1:
    fpl_previous = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek-1}.csv')
    new_ids = set(fpl_data['id']).difference(set(fpl_previous['id']))
    old_ids = set(fpl_data['id']).intersection(set(fpl_previous['id']))
    # convert all difference columns to floats
    fpl_data[difference_columns] = fpl_data[difference_columns].astype(float)
    fpl_gameweek = fpl_data.apply(lambda x: calculate_difference_to_previous_gameweek(x), axis=1)
    fpl_gameweek = fpl_gameweek[fpl_gameweek['minutes']>0]
    print(f'Number of players that played this week: {fpl_gameweek.shape[0]}')
    fpl_previous_weeks = pd.read_csv('../data/fpl_data.csv', index_col=0)
    print(f'Size of fpl_previous_weeks: {fpl_previous_weeks.shape[0]}')
    #fpl_previous_weeks = fpl_previous_weeks[fpl_previous_weeks['GW']<gameweek]
    fpl_all_weeks = pd.concat([fpl_previous_weeks, fpl_gameweek], axis=0)
    print(f'Size of fpl_all_weeks: {fpl_all_weeks.shape[0]}')
    display(fpl_all_weeks)
else:
    print('Check gameweek!')

In [ ]:
# save data
if gameweek>1:
    fpl_all_weeks.to_csv('../data/fpl_data.csv')

# Fetch fbref data

In [ ]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
links = download_data.get_match_report_links(url)
links

In [ ]:
# check which matches (links) are already collected
with open('../data/fbref_fixture_links.txt') as f:
    old_links = f.read().splitlines()
old_links

In [ ]:
# get links to games for which data has not been collected yet
new_links = [link for link in links if link not in old_links]
new_links

In [ ]:
fbref_data = download_data.download_match_report_data(new_links)
# normalize spelling of player names
fbref_data[('General','Player')] = fbref_data[('General','Player')].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
# map certain player names such that they are the same for both fpl and fbref data
#fbref_data[('General','Player')] = fbref_data[('General','Player')].map(player_name_dict).fillna(fbref_data[('General','Player')])
fbref_data

In [ ]:
# change column name to align the new Opta data with the old Statsbomb data
fbref_data.columns = fbref_data.columns.values
fbref_data.columns = pd.MultiIndex.from_tuples(fbref_data.rename(columns={('Expected', 'xAG'): ('Expected', 'xA')}))
fbref_data

In [ ]:
# save links that have been downloaded already so that in the future one can only fetch data from new games
with open('../data/fbref_fixture_links.txt', 'w') as f:
    f.write('\n'.join(links))

In [ ]:
old_fbref_data = pd.read_csv('../data/fbref_data.csv', index_col=0, header=[0,1])
#old_fbref_data.to_csv(f'../data/fbref_data_backup_week{gameweek}.csv')

In [ ]:
fbref_data = pd.concat([old_fbref_data, fbref_data], axis=0)
print(f'Number of rows in fbref_data: {fbref_data.shape[0]}.')

In [ ]:
fbref_data.to_csv(f'../data/fbref_data.csv')

## 22-23 Match results

In [ ]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
data = pd.read_html(url)
results = data[0].dropna(axis=0, how='all').copy()
results

In [ ]:
# fix Nott Forest team name
my_dict = {"Nott'ham Forest":'Nottingham Forest'}
results['Home'] = results['Home'].map(my_dict).fillna(results['Home'])
results['Away'] = results['Away'].map(my_dict).fillna(results['Away'])

In [ ]:
results.to_csv('../data/PL_results_season22_23.csv')